In [2]:
# ===================================================================
# ЯЧЕЙКА 1: ПОДКЛЮЧЕНИЕ К GOOGLE DRIVE И НАЧАЛЬНАЯ НАСТРОЙКА
# ===================================================================

# 1. Подключаем ваш Google Drive
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# 2. Устанавливаем все необходимые библиотеки
# Добавляем requests для работы с OpenRouter API
!pip install pandas pyarrow tqdm requests -q

# 3. Определяем ключевые пути и параметры
# ВАЖНО: Измените этот путь на тот, где в вашем Google Drive лежит папка 'data'
GDRIVE_PROJECT_PATH = Path('/content/drive/MyDrive/shitcoin_analyze/') # <--- ИЗМЕНИТЕ ЭТОТ ПУТЬ

TWEETS_DATA_DIR = GDRIVE_PROJECT_PATH / "data_uranus/"
# Итоговый файл будет сохранен в корневой папке вашего проекта на Google Drive
OUTPUT_PARQUET_FILE = GDRIVE_PROJECT_PATH / "all_tweets_annotated_uranus.parquet"

print(f"Рабочая директория установлена: {GDRIVE_PROJECT_PATH}")
print(f"Файлы с твитами будут искаться в: {TWEETS_DATA_DIR}")
print(f"Итоговый файл будет сохранен как: {OUTPUT_PARQUET_FILE}")

Mounted at /content/drive
Рабочая директория установлена: /content/drive/MyDrive/shitcoin_analyze
Файлы с твитами будут искаться в: /content/drive/MyDrive/shitcoin_analyze/data_uranus
Итоговый файл будет сохранен как: /content/drive/MyDrive/shitcoin_analyze/all_tweets_annotated_uranus.parquet


In [3]:
OUTPUT_PARQUET_FILE

PosixPath('/content/drive/MyDrive/shitcoin_analyze/all_tweets_annotated_uranus.parquet')

In [5]:
# ===================================================================
# ЯЧЕЙКА 3: ШАГ 1 - ОБЪЕДИНЕНИЕ ВСЕХ ТВИТОВ В ОДИН DATAFRAME
# ===================================================================
import pandas as pd
from pathlib import Path
import json

def combine_tweets_from_json(data_dir: Path) -> pd.DataFrame:
    if not data_dir.exists():
        print(f"Ошибка: Директория не найдена: {data_dir}")
        return pd.DataFrame()

    all_tweets_list = []
    json_files = list(data_dir.glob("*.json"))
    if not json_files:
        print(f"В директории {data_dir} не найдено JSON-файлов.")
        return pd.DataFrame()

    print(f"Найдено {len(json_files)} JSON-файлов для обработки...")

    for filepath in json_files:
        username = filepath.stem
        with open(filepath, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                tweets_in_file = data.get("tweets", [])
                for tweet in tweets_in_file:
                    tweet['author_username'] = username
                    all_tweets_list.append(tweet)
            except Exception as e:
                print(f"Проблема с файлом {filepath}: {e}")

    if not all_tweets_list:
        print("Не удалось загрузить ни одного твита.")
        return pd.DataFrame()

    df = pd.DataFrame(all_tweets_list)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    numeric_cols = ['replies', 'reposts', 'likes', 'views']
    for col in numeric_cols:
        if col not in df.columns: df[col] = 0
    df[numeric_cols] = df[numeric_cols].fillna(0).astype(int)

    # Удаляем дубликаты по тексту твита, оставляя первый встреченный
    df.drop_duplicates(subset=['text'], inplace=True, keep='first')
    df.reset_index(drop=True, inplace=True)

    print(f"Обработка завершена. Всего собрано {len(df)} уникальных твитов.")
    return df

# Запускаем процесс объединения
main_df = combine_tweets_from_json(TWEETS_DATA_DIR)

if not main_df.empty:
    print("\nПример первых 5 строк объединенного датафрейма:")
    display(main_df.head())

Найдено 88 JSON-файлов для обработки...
Обработка завершена. Всего собрано 4636 уникальных твитов.

Пример первых 5 строк объединенного датафрейма:


,parse_timestamp,text,timestamp,url,replies,reposts,likes,views,author_username
0,2025-09-03T19:24:52.837017+00:00,"NYC, next stop uranus! \n\nuranus is going to ...",2025-09-02 22:16:12+00:00,https://x.comhttps://x.com/Enter_Uranus/status...,21,94,304,13787,AdagioUday
1,2025-09-03T19:24:52.854446+00:00,Good to have you back,2025-09-02 21:35:53+00:00,https://x.comhttps://x.com/jup_studio/status/1...,25,32,183,6313,AdagioUday
2,2025-09-03T19:24:52.895081+00:00,$URANUS main account \n@Enter_Uranus\n has bee...,2025-09-02 17:29:30+00:00,https://x.comhttps://x.com/AdagioUday/status/1...,0,0,0,81,AdagioUday
3,2025-09-03T19:24:55.716377+00:00,So \n@JackMa\n is buying $ETH at 4400 and you ...,2025-09-02 17:12:32+00:00,https://x.comhttps://x.com/AdagioUday/status/1...,0,0,0,64,AdagioUday
4,2025-09-03T19:24:52.876843+00:00,"Jack Ma’s Yunfeng Financial bought $44,000,000...",2025-09-02 16:30:49+00:00,https://x.comhttps://x.com/TedPillows/status/1...,94,83,669,26786,AdagioUday


In [6]:
# ===================================================================
# ЯЧЕЙКА 4 (v2): АНСАМБЛИРОВАННАЯ АННОТАЦИЯ ТВИТОВ (5 ВЕРСИЙ + УСРЕДНЕНИЕ) С OPENROUTER
# ===================================================================
import numpy as np
import pandas as pd
import json
import time
import requests
from tqdm import tqdm
import os

# --- Конфигурация для этого шага ---
BATCH_SIZE = 50
ANNOTATION_VERSIONS = 5  # Количество разметок для каждого батча
MAX_RETRIES_PER_VERSION = 3  # Количество попыток для КАЖДОЙ из 5 версий
SAVE_EVERY_N_BATCHES = 1  # Сохраняем после КАЖДОГО обработанного батча для макс. надежности

# Получаем API ключ OpenRouter
# Замените 'YOUR_OPENROUTER_API_KEY' на ваш реальный ключ
# Вы можете добавить его в Colab через:
#   from google.colab import userdata
#   OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
# Или установить как переменную окружения
OPENROUTER_API_KEY = ''

# Ваш системный промпт (без изменений)
SYSTEM_INSTRUCTION = """# ROLE
You are a precise crypto tweet classifier. Output ONLY a valid JSON array with coordinate scores for each tweet.

# AXES DEFINITION (CRITICAL - USE THESE EXACT SCALES)
- X-axis: [Promotion-Orientation] [0.00, 1.00]
  • Measures intent: from educational (0.00) to promotional (1.00)
  • 0.00 = Pure knowledge sharing ("tutorial", "explained", "how it works")
  • 0.50 = Neutral information ("ETH up 5%", "event update")
  • 1.00 = Direct promotion ("BUY", "MOON", "1000x", "GEM", "ALERT")
- Y-axis: [Analysis-Depth] [0.00, 1.00]
  • Measures analytical depth: from surface (0.00) to deep (1.00)
  • 0.00 = No analysis ("GM", "HODL", price move without context)
  • 1.00 = In-depth analysis (charts, on-chain data, tokenomics, code)
- Z-axis: [Emotional-Intensity] [0.00, 1.00]
  • Measures emotional tone: from robotic (0.00) to viral (1.00)
  • 0.00 = Machine-like neutrality ("Data shows...", "Report:...")
  • 0.50 = Normal conversational tone
  • 1.00 = High virality ("!!!", "FOMO", "SCAM ALERT", fire/moon emojis)
- T-axis: [Cryptocurrency-Relevance] [0.00, 1.00]
  • Measures how relevance for cryptocurrency: from irrelevant (0.00) to relevant (1.00)
  • 0.00 = Text is not relevant to cryptocurrency at all
  • 1.00 = Text is about cryptocurrency

# INPUT FORMAT
Each tweet is wrapped in:
<TWEET>
content
</TWEET>

# OUTPUT RULES
1. For EACH tweet, output: {"id": "TWEET_ID", "x": FLOAT, "y": FLOAT, "z": FLOAT, "t": FLOAT}
2. Use FLOAT values in the EXACT ranges above
3. ALL values MUST have **exactly 2 decimal places** (e.g., 0.92, 0.45, 0.00 — NOT 0.9, 1, or 0.923)
4. NEVER add text before or after JSON
5. If tweet is empty/irrelevant: {"id": "TWEET_ID", "x": 0.00, "y": 0.00, "z": 0.00, "t": 0.00}

# EXAMPLE
Input:
<TWEET>
1. "SOL 1000x GEM!!! BUY NOW 🔥"
</TWEET>
<TWEET>
2. "Let's break down Uniswap v3 fees and LP risks [thread]"
</TWEET>

Output:
[
  {"id": "1", "x": 0.95, "y": 0.45, "z": 0.96, "t": 0.98},
  {"id": "2", "x": 0.10, "y": 0.75, "z": 0.50, "t": 0.95}
]"""

# --- Вспомогательные функции ---
def prepare_batch_prompt(batch_df: pd.DataFrame) -> str:
    prompt_parts = []
    for index, row in batch_df.iterrows():
        # Используем реальный индекс DataFrame как ID, это самый надежный способ
        tweet_id = index
        tweet_text = str(row['text']).replace('<', '').replace('>', '')
        prompt_parts.append(f"<TWEET>\n{tweet_id}. \"{tweet_text}\"\n</TWEET>")
    return "\n".join(prompt_parts)

def get_one_annotation_version(openrouter_api_key, batch_df: pd.DataFrame, version_num: int) -> list | None:
    """Делает ОДНУ попытку аннотации с внутренними ретраями через OpenRouter API."""
    prompt = prepare_batch_prompt(batch_df)

    # Формируем сообщения в формате OpenAI Chat API
    messages = [
        {"role": "system", "content": SYSTEM_INSTRUCTION},
        {"role": "user", "content": prompt}
    ]

    for attempt in range(MAX_RETRIES_PER_VERSION):
        try:
            tqdm.write(f"  Версия {version_num}, попытка {attempt + 1}/{MAX_RETRIES_PER_VERSION}...")

            # Отправляем запрос к OpenRouter API
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {openrouter_api_key}",
                    "Content-Type": "application/json",
                    # Опционально: добавляем заголовки для идентификации приложения
                    #"HTTP-Referer": "https://colab.research.google.com",
                    #"X-Title": "Crypto Tweet Analyzer"
                },
                json={
                    "model": "z-ai/glm-4.5-air:free",  # Используем правильный формат: организация/модель
                    "messages": messages,
                    "temperature": 0.7, # Запрашиваем структурированный JSON
                    "reasoning": {
                        "enabled": False
                    }
                }
            )

            # Проверяем статус ответа
            if response.status_code != 200:
                error_msg = response.text
                tqdm.write(f"    ❌ Ошибка API: {response.status_code} - {error_msg}")
                continue

            response_data = response.json()

            # Извлекаем ответ из ответа API
            response_text = response_data["choices"][0]["message"]["content"].strip()
            #print(response_text)
            # Удаляем возможные маркеры кода
            if response_text.startswith("```json"):
                response_text = response_text[7:-3].strip()

            # Парсим JSON
            annotations = json.loads(response_text)

            # Проверяем, что количество аннотаций соответствует количеству твитов в батче
            if len(annotations) == len(batch_df):
                return annotations  # Успех!
            else:
                tqdm.write(f"    ⚠️ Ошибка валидации: модель вернула {len(annotations)}/{len(batch_df)} аннотаций.")

        except Exception as e:
            tqdm.write(f"    ❌ Ошибка API или парсинга: {e}")

        # Увеличивающаяся задержка перед повторной попыткой
        time.sleep(3 * (attempt + 1))

    tqdm.write(f"  ⛔ Не удалось получить версию {version_num} для этого батча.")
    return None

# --- Основной асинхронный процесс ---
def run_annotation_process(df: pd.DataFrame):
    annotation_cols = ['promo_score_x', 'depth_score_y', 'emotion_score_z', 'relevance_score_t']
    for col in annotation_cols:
        if col not in df.columns: df[col] = np.nan

    to_annotate_df = df[df['promo_score_x'].isnull()].copy()
    if to_annotate_df.empty:
        print("🎉 Все твиты уже аннотированы!")
        return
    print(f"Необходимо аннотировать {len(to_annotate_df)} твитов.")

    batches = [to_annotate_df.iloc[i:i + BATCH_SIZE] for i in range(0, len(to_annotate_df), BATCH_SIZE)]

    # Основной цикл по батчам
    for i, batch_df in enumerate(tqdm(batches, desc="Обработка батчей")):
        tqdm.write(f"\n--- Обработка батча {i + 1}/{len(batches)} (индексы {batch_df.index.min()}-{batch_df.index.max()}) ---")

        # 1. Собираем 5 версий аннотации
        successful_versions = []
        for version in range(1, ANNOTATION_VERSIONS + 1):
            result = get_one_annotation_version(OPENROUTER_API_KEY, batch_df, version)
            if result:
                successful_versions.append(result)

        tqdm.write(f"  Получено {len(successful_versions)}/{ANNOTATION_VERSIONS} успешных версий для батча.")

        # 2. Если есть хотя бы одна успешная версия, усредняем и сохраняем
        if successful_versions:
            # Агрегируем все результаты по ID твита
            aggregated_scores = {}
            for version_result in successful_versions:
                for ann in version_result:
                    try:
                        # ID - это индекс в DataFrame, который мы передали в промпте
                        tweet_id = int(ann['id'])
                        if tweet_id not in aggregated_scores:
                            aggregated_scores[tweet_id] = {'x': [], 'y': [], 'z': [], 't': []}

                        aggregated_scores[tweet_id]['x'].append(float(ann['x']))
                        aggregated_scores[tweet_id]['y'].append(float(ann['y']))
                        aggregated_scores[tweet_id]['z'].append(float(ann['z']))
                        aggregated_scores[tweet_id]['t'].append(float(ann['t']))
                    except (ValueError, KeyError, TypeError):
                        continue  # Пропускаем кривые аннотации

            # 3. Усредняем и записываем в основной DataFrame
            for tweet_id, scores in aggregated_scores.items():
                if tweet_id in df.index:
                    df.loc[tweet_id, 'promo_score_x'] = np.mean(scores['x'])
                    df.loc[tweet_id, 'depth_score_y'] = np.mean(scores['y'])
                    df.loc[tweet_id, 'emotion_score_z'] = np.mean(scores['z'])
                    df.loc[tweet_id, 'relevance_score_t'] = np.mean(scores['t'])

            # 4. Сохраняем прогресс
            df.to_parquet(OUTPUT_PARQUET_FILE, index=False)
            tqdm.write(f"💾 Прогресс сохранен. Батч {i + 1} усреднен и записан в файл.")

# Запускаем аннотацию
if 'main_df' in locals() and not main_df.empty:
    if OUTPUT_PARQUET_FILE.exists():
        print(f"Найден существующий файл {OUTPUT_PARQUET_FILE}. Загружаем его для продолжения работы.")
        main_df = pd.read_parquet(OUTPUT_PARQUET_FILE)

    run_annotation_process(main_df)
else:
    print("DataFrame 'main_df' не был создан. Выполните Ячейку 3.")

Необходимо аннотировать 4636 твитов.


Обработка батчей:   0%|          | 0/93 [00:00<?, ?it/s]


--- Обработка батча 1/93 (индексы 0-49) ---
  Версия 1, попытка 1/3...


Обработка батчей:   0%|          | 0/93 [01:20<?, ?it/s]

  Версия 2, попытка 1/3...


...

  Версия 4, попытка 1/3...


Обработка батчей:  99%|█████████▉| 92/93 [10:04:32<06:11, 371.26s/it]

  Версия 5, попытка 1/3...


Обработка батчей: 100%|██████████| 93/93 [10:05:44<00:00, 390.81s/it]

  Получено 5/5 успешных версий для батча.
💾 Прогресс сохранен. Батч 93 усреднен и записан в файл.


In [ ]:
main_df = pd.read_parquet(OUTPUT_PARQUET_FILE)